In [1]:
import re
import sys
import h5py
import time
import numpy as np
import scipy as sp
from numba import jit
import matplotlib.pyplot as plt
from pyXSteam.XSteam import XSteam
#from scipy.interpolate import interp1d
#from scipy.sparse import coo_matrix


In [2]:
"""# Global HDF5 structure
global_g = None

def create_hdf5_structure():
    global global_g
    with h5py.File("data.h5", "w") as file:
        group = file.create_group("g")
        group.attrs["nz"] = 10
        global_g = group

def access_hdf5_structure():
    global global_g
    # Access and use the global HDF5 structure
    if global_g is not None:
        with h5py.File("data.h5", "r") as file:
            nz = file["g"].attrs["nz"]
            print("nz =", nz)

create_hdf5_structure()
access_hdf5_structure()
"""

'# Global HDF5 structure\nglobal_g = None\n\ndef create_hdf5_structure():\n    global global_g\n    with h5py.File("data.h5", "w") as file:\n        group = file.create_group("g")\n        group.attrs["nz"] = 10\n        global_g = group\n\ndef access_hdf5_structure():\n    global global_g\n    # Access and use the global HDF5 structure\n    if global_g is not None:\n        with h5py.File("data.h5", "r") as file:\n            nz = file["g"].attrs["nz"]\n            print("nz =", nz)\n\ncreate_hdf5_structure()\naccess_hdf5_structure()\n'

In [3]:
# Global variables
#global g, th, fr

# Function to specify the geometry and nodalization of the fuel rod
#def input_and_initialize_PWR_like():
#global g, th, fr
# Global HDF5 structure
#global_g = None

def create_hdf5_structure():
    #global global_g
    with h5py.File("input_and_initialize_PWR_like.h5", "w") as hdf:
        g  = hdf.create_group("g")
        th = hdf.create_group("th")
        fr = hdf.create_group("fr")

        #group.attrs["nz"] = 10
        #global_g = group

        # Input fuel rod geometry and nodalization
        g_nz = 10  # number of axial nodes
        g.create_dataset("nz", data=g_nz)

        g_fuel_rIn = 0  # inner fuel radius (m)
        g_fuel_rOut = 4.12e-3  # outer fuel radius (m)
        g_fuel_nr = 20  # number of radial nodes in fuel
        g_fuel = g.create_group("fuel")
        g_fuel.create_dataset("rIn",  data=g_fuel_rIn)
        g_fuel.create_dataset("rOut", data=g_fuel_rOut)
        g_fuel.create_dataset("nr",   data=g_fuel_nr)

        g_clad_rIn = 4.22e-3  # inner clad radius (m)
        g_clad_rOut = 4.75e-3  # outer clad radius (m)
        g_clad_nr = 5
        g_clad = g.create_group("clad")
        g_clad.create_dataset("rIn",  data=g_clad_rIn)
        g_clad.create_dataset("rOut", data=g_clad_rOut)
        g_clad.create_dataset("nr",   data=g_clad_nr)

        g_cool_pitch = 13.3e-3  # square unit cell pitch (m)
        g_cool_rOut = np.sqrt(g_cool_pitch**2 / np.pi)  # equivalent radius of the unit cell (m)
        g_cool = g.create_group("cool")
        g_cool.create_dataset("pitch", data=g_cool_pitch)
        g_cool.create_dataset("rOut",  data=g_cool_rOut)

        g_dz0 = 0.3 * np.ones(g_nz)  # height of the node (m)
        g_dzGasPlenum = 0.2  # height of the fuel rod gas plenum assuming it is empty (m)
        g.create_dataset("dz0", data = g_dz0)
        g.create_dataset("dzGasPlenum", data = g_dzGasPlenum)

        # Input average power rating in fuel
        th_qLHGR0 = np.array([  [0, 10, 1e20],  # time (s)
                                [200e2, 200e2, 200e2]   ])  # linear heat generation rate (W/m)
        th.create_dataset("qLHGR0", data = th_qLHGR0)

        # Input fuel rod parameters
        fr_clad_fastFlux = np.array([   [0, 10, 1e20],  # time (s)
                                        [1e13, 1e13, 1e13]  ])  # fast flux in cladding (1/cm2-s)
        fr_clad = fr.create_group("clad")
        fr_clad.create_dataset("fastFlux", data = fr_clad_fastFlux)

        fr_fuel_FGR = np.array([[0, 10, 1e20],  # time (s)
                                [0.03, 0.03, 0.03]])  # fission gas release (-)
        fr_fuel = fr.create_group("fuel")
        fr_fuel.create_dataset("FGR", data = fr_fuel_FGR)

        fr_ingas_Tplenum = 533  # fuel rod gas plenum temperature (K)
        fr_ingas_p0 = 1  # as-fabricated helium pressure inside fuel rod (MPa)
        fr_fuel_por = 0.05 * np.ones((g_nz, g_fuel_nr))  # initial fuel porosity (-)
        fr_ingas = fr.create_group("ingas")
        fr_ingas.create_dataset("Tplenum", data = fr_ingas_Tplenum)
        fr_ingas.create_dataset("p0",      data = fr_ingas_p0)
        fr_fuel.create_dataset("por",      data = fr_fuel_por)

        # Input channel geometry
        g_aFlow = 8.914e-5 * np.ones(g_nz)  # flow area (m2)
        g.create_dataset("aFlow", data = g_aFlow)

        # Input channel parameters
        th_mdot0_ = np.array([  [0, 10, 1000],  # time (s)
                                [0.3, 0.3, 0.3]])  # flowrate (kg/s) 0.3
        th_p0 = 16  # coolant pressure (MPa)
        th_T0 = 533.0  # inlet temperature (K)
        th.create_dataset("mdot0", data = th_mdot0_)
        th.create_dataset("p0",    data = th_p0)
        th.create_dataset("T0",    data = th_T0)

        # Initialize fuel geometry
        g_fuel_dr0 = (g_fuel_rOut - g_fuel_rIn) / (g_fuel_nr - 1)  # fuel node radial thickness (m)
        g_fuel_r0 = np.arange(g_fuel_rIn, g_fuel_rOut + g_fuel_dr0, g_fuel_dr0)  # fuel node radius (m)
        g_fuel_r0_ = np.concatenate(([g_fuel_rIn], np.interp(np.arange(1.5, g_fuel_nr + 0.5), np.arange(1, g_fuel_nr + 1),
                                                                    g_fuel_r0), [g_fuel_rOut]))  # fuel node boundary (m)
        g_fuel_a0_ = np.transpose(np.tile(2*np.pi*g_fuel_r0_[:, None], g_nz)) * np.tile(g_dz0[:, None], (1, g_fuel_nr + 1))  # XS area of fuel node boundary (m2)
        g_fuel_v0 = np.transpose(np.tile(np.pi*np.diff(g_fuel_r0_**2)[:, None], g_nz)) * np.tile(g_dz0[:, None], (1, g_fuel_nr))  # fuel node volume (m3)
        g_fuel_vFrac = (g_fuel_rOut**2 - g_fuel_rIn**2) / g_cool_rOut**2
        g_fuel.create_dataset("dr0",   data = g_fuel_dr0)
        g_fuel.create_dataset("r0",    data = g_fuel_r0)
        g_fuel.create_dataset("r0_",   data = g_fuel_r0_)
        g_fuel.create_dataset("a0_",   data = g_fuel_a0_)
        g_fuel.create_dataset("v0",    data = g_fuel_v0)
        g_fuel.create_dataset("vFrac", data = g_fuel_vFrac)

        # Initialize clad geometry
        g_clad_dr0 = (g_clad_rOut - g_clad_rIn) / (g_clad_nr - 1)  # clad node radial thickness (m)
        g_clad_r0 = np.arange(g_clad_rIn, g_clad_rOut + g_clad_dr0, g_clad_dr0)  # clad node radius (m)
        g_clad_r0_ = np.concatenate(([g_clad_rIn], np.interp(np.arange(1.5, g_clad_nr + 0.5), np.arange(1, g_clad_nr + 1), 
                                                                    g_clad_r0), [g_clad_rOut]))  # clad node boundary (m)
        g_clad_a0_ = np.transpose(np.tile(2 * np.pi * g_clad_r0_[:, None], g_nz)) * np.tile(g_dz0[:, None], (1, g_clad_nr + 1))  # XS area of clad node boundary (m2)
        g_clad_v0 = np.transpose(np.tile(np.pi*np.diff(g_clad_r0_**2)[:, None], g_nz)) * np.tile(g_dz0[:, None], (1, g_clad_nr))  # clad node volume (m3)
        g_clad_vFrac = (g_clad_rOut**2 - g_clad_rIn**2) / g_cool_rOut**2
        g_clad.create_dataset("dr0",   data = g_clad_dr0)
        g_clad.create_dataset("r0",    data = g_clad_r0)
        g_clad.create_dataset("r0_",   data = g_clad_r0_)
        g_clad.create_dataset("a0_",   data = g_clad_a0_)
        g_clad.create_dataset("v0",    data = g_clad_v0)
        g_clad.create_dataset("vFrac", data = g_clad_vFrac)

        # Initialize gap geometry
        dimensions = tuple(range(1, g_nz+1))
        g_gap_dr0 = (g_clad_rIn - g_fuel_rOut) * np.ones(dimensions)   # initial cold gap (m)
        g_gap_r0_ = (g_clad_rIn + g_fuel_rOut) / 2  # average gap radius (m)
        g_gap_a0_ = (2 * np.pi * g_gap_r0_ * np.ones((g_nz, 1))) * g_dz0  # XS area of the mid-gap (m2)
        g_gap_vFrac = (g_clad_rIn**2 - g_fuel_rOut**2) / g_cool_rOut**2
        g_gap = g.create_group("gap")
        g_gap.create_dataset("dr0",   data = g_gap_dr0.flatten())
        g_gap.create_dataset("r0_",   data = g_gap_r0_)
        g_gap.create_dataset("a0_",   data = g_gap_a0_)
        g_gap.create_dataset("vFrac", data = g_gap_vFrac)

        # Initialize as-fabricated inner volumes and gas amount
        g_vGasPlenum = g_dzGasPlenum * np.pi * g_clad_rIn**2  # gas plenum volume (m3)
        g_vGasGap = g_dz0 * np.pi * (g_clad_rIn**2 - g_fuel_rOut**2)  # gas gap volume (m3)
        g_vGasCentralVoid = g_dz0 * np.pi * g_fuel_rIn**2  # gas central void volume (m3)
        fr_ingas_muHe0 = fr_ingas_p0 * (g_vGasPlenum + np.sum(g_vGasGap + g_vGasCentralVoid)) / (8.31e-6 * 293)  # as-fabricated gas amount inside fuel rod (mole)
        g.create_dataset("vGasPlenum",      data = g_vGasPlenum)
        g.create_dataset("vGasGap",         data = g_vGasGap)
        g.create_dataset("vGasCentralVoid", data = g_vGasCentralVoid)
        fr_ingas.create_dataset("muHe0",    data = fr_ingas_muHe0)

        # Initialize gas gap status
        g_gap_open = np.ones(g_nz)
        g_gap_clsd = np.zeros(g_nz)
        g_gap.create_dataset("open", data = g_gap_open)
        g_gap.create_dataset("clsd", data = g_gap_clsd)

        # Initialize fuel and clad total deformation components
        fr_fuel_eps0 = np.zeros((3, g_nz, g_fuel_nr))
        fr_clad_eps0 = np.zeros((3, g_nz, g_clad_nr))
        fuel_eps0 = fr_fuel.create_group("eps0")
        clad_eps0 = fr_clad.create_group("eps0")
        for i in range(3):
            fr_fuel_eps0[i] = np.zeros((g_nz, g_fuel_nr))
            fr_clad_eps0[i] = np.zeros((g_nz, g_clad_nr))
            fuel_eps0.create_dataset(f"eps0(0,{i})", data = fr_fuel_eps0[i])
            clad_eps0.create_dataset(f"eps0(0,{i})", data = fr_clad_eps0[i])

        # Initialize flow channel geometry
        g_volFlow = g_aFlow * g_dz0  # volume of node (m3)
        g_areaHX = 2 * np.pi * g_clad_rOut * g_dz0  # heat exchange area(m2)(m2)
        g_dHyd = 4 * g_volFlow / g_areaHX  # hydraulic diameter (m)
        g_cool_vFrac = (g_cool_rOut**2 - g_clad_rOut**2) / g_cool_rOut**2
        g.create_dataset("volFlow",    data = g_volFlow)
        g.create_dataset("areaHX",     data = g_areaHX)
        g.create_dataset("dHyd",       data = g_dHyd)
        g_cool.create_dataset("vFrac", data = g_cool_vFrac)

        # Initialize thermal hydraulic parameters
        # Path to steam-water properties
        steamTable = XSteam(XSteam.UNIT_SYSTEM_MKS)
        th_h0 = steamTable.h_pt(th_p0 / 10, th_T0 - 273) * 1e3  # water enthalpy at core inlet (J/kg)
        th_h = np.ones(g_nz) * th_h0  # initial enthalpy in nodes (kJ/kg)
        th_p = np.ones(g_nz) * th_p0  # initial pressure in nodes (MPa)
        th.create_dataset("h0", data = th_h0)
        th.create_dataset("h", data = th_h)
        th.create_dataset("p", data = th_p)
create_hdf5_structure()

In [4]:
with h5py.File("input_and_initialize_PWR_like.h5", "r") as f:
    #print(list(f.get("g").get("gap").keys()))
    dr0_test = np.array(f.get("g").get("gap").get("dr0"))
    print(dr0_test.shape)
    dr0_reshaped = dr0_test.reshape(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
    print(dr0_reshaped.shape)

(3628800,)
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


In [5]:
np.math.factorial(10)

3628800

In [6]:
#print(np.transpose(np.tile(2 * np.pi * g_clad_r0_[:, None], g_nz)))
#print((2*np.pi*g_fuel_r0_[:, None] * np.ones((1, g_nz))).shape)
#print(np.tile(g_dz0[:, None], (1, g_fuel_nr + 1)).shape)
#print(((g_clad_rIn - g_fuel_rOut) @ np.ones(g_nz)).shape)

dimensions = tuple(range(1, 11))    #(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
print(dimensions)
ans = np.ones(dimensions)
ans.shape

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [7]:
"""import numpy as np

# Global structure with geometry parameters
#g = {}

#def createPWR_like_mix():
#global g

#PWRmix = {}
PWRmix_ng = 421

# Path to library
sys.path.append('..\\00.Lib')

# Input and initialize the geometry of the PWR-like unit cell
from input_and_initialize_PWR_like import input_and_initialize_PWR_like
input_and_initialize_PWR_like()

# Path to microscopic cross section data
sys.path.append('..\\01.Micro.XS.421g')

# Call the functions for UO2 isotopes and store the data in the structures
from micro_U_235__600K import micro_U_235__600K
from micro_U_238__600K import micro_U_238__600K
from micro_O_016__600K import micro_O_016__600K
U235 = micro_U_235__600K()
U238 = micro_U_238__600K()
O16 = micro_O_016__600K()
PWRmix['eg'] = U235['eg']

# UO2 ceramic fuel is manufactured with the density lower than the theoretical density
por = 0.05

# Uranium is composed of two uranium isotopes: U235 and U238
molEnrich = 0.03

# The molar fractions of U235 and U238
molFrU = np.zeros(2)
molFrU[0] = molEnrich
molFrU[1] = 1 - molFrU[0]

# Mass of one "average" UO2 molecule in atomic unit mass [a.u.m.]
UO2_03 = {}
UO2_03['aw'] = U235['aw'] * molFrU[0] + U238['aw'] * molFrU[1] + O16['aw'] * 2.0

# Path to material properties
sys.path.append('..\\00.Lib')
from matpro import matpro
_, _, fuel = matpro([], [], {})

# The UO2 fuel density is theoretical density times 1 - porosity
UO2_03['den'] = fuel['rho'] * 1e-3 * (1 - por)  # [g/cm3]
rho = UO2_03['den'] * 1.0e-24  # [g/(barn*cm)]
rho = rho / 1.660538e-24  # [(a.u.m.)/(barn*cm)]
rho = rho / UO2_03['aw']  # [number of UO2 molecules/(barn*cm)]

# The names of fissionable isotopes and oxygen
UO2_03['isoName'] = ['U235', 'U238', 'O16']

# The number densities of fissionable isotopes and oxygen
UO2_03['numDen'] = np.zeros(3)
UO2_03['numDen'][0] = molFrU[0] * rho * g['fuel']['vFrac']
UO2_03['numDen'][1] = molFrU[1] * rho * g['fuel']['vFrac']
UO2_03['numDen'][2] = 2 * rho * g['fuel']['vFrac']

   
"""

'import numpy as np\n\n# Global structure with geometry parameters\n#g = {}\n\n#def createPWR_like_mix():\n#global g\n\n#PWRmix = {}\nPWRmix_ng = 421\n\n# Path to library\nsys.path.append(\'..\\00.Lib\')\n\n# Input and initialize the geometry of the PWR-like unit cell\nfrom input_and_initialize_PWR_like import input_and_initialize_PWR_like\ninput_and_initialize_PWR_like()\n\n# Path to microscopic cross section data\nsys.path.append(\'..\\01.Micro.XS.421g\')\n\n# Call the functions for UO2 isotopes and store the data in the structures\nfrom micro_U_235__600K import micro_U_235__600K\nfrom micro_U_238__600K import micro_U_238__600K\nfrom micro_O_016__600K import micro_O_016__600K\nU235 = micro_U_235__600K()\nU238 = micro_U_238__600K()\nO16 = micro_O_016__600K()\nPWRmix[\'eg\'] = U235[\'eg\']\n\n# UO2 ceramic fuel is manufactured with the density lower than the theoretical density\npor = 0.05\n\n# Uranium is composed of two uranium isotopes: U235 and U238\nmolEnrich = 0.03\n\n# The molar 